# Agente com Gemini + System Prompt via `prefix`

Este notebook mostra como usar o modelo Gemini com `initialize_agent`, adicionando um system prompt corretamente via `agent_kwargs['prefix']`.

## FLUXOGRAMA
                          +---------------------+
                          | Início do Processo  |
                          +----------+----------+
                                     |
                                     v
                     +---------------+----------------+
                     | Pergunta do aluno é definida   |
                     +---------------+----------------+
                                     |
                                     v
         +--------------------------+---------------------------+
         | Agente LangChain (tutor) responde usando Gemini      |
         | - Usa memória de conversa                            |
         | - Segue prompt de tutor                              |
         +--------------------------+---------------------------+
                                     |
                                     v
                +--------------------+--------------------+
                | Resposta do tutor armazenada e exibida  |
                +--------------------+--------------------+
                                     |
                                     v
                   +----------------+----------------+
                   | Juiz (novo modelo Gemini)       |
                   | - Temperatura mais baixa        |
                   | - Segue prompt crítico          |
                   +----------------+----------------+
                                     |
                                     v
          +--------------------------+---------------------------+
          | Função avaliar_resposta é chamada                    |
          | - Junta pergunta e resposta do tutor                 |
          | - Envia para o juiz avaliar com base nos critérios   |
          +--------------------------+---------------------------+
                                     |
                                     v
                       +---------------------------+
                       | Juiz retorna avaliação:   |
                       |  Aprovado ou  Reprovado   |
                       +---------------------------+
                                     |
                                     v
                        +-----------------------+
                        | Exibe resultado final |
                        +-----------+-----------+
                                    |
                                    v
                            +-------+------+
                            | Fim do Fluxo |
                            +--------------+



## **Instalação das bibliotecas**
* `langchain`: orquestrador de fluxos com LLMs (Large Language Models).
* `langchain-google-genai`: integração com o modelo **Gemini** (Google).


In [23]:
# Instale as bibliotecas necessárias se ainda não tiver
# !pip install langchain langchain-google-genai

## **Imports e ambiente**
* `from langchain_google_genai import ChatGoogleGenerativeAI`

 	Este comando importa a classe que conecta o LangChain ao modelo Gemini do Google.
    Você usa essa classe para criar um objeto de conversa com IA (como um chatbot ou tutor), definindo o modelo (gemini-2.0, por exemplo), temperatura, e chave de API.
* `from langchain.memory import ConversationBufferMemory`

	Importa a classe responsável por guardar o histórico das conversas entre o usuário e o modelo, permitindo que o agente “lembre” do que foi dito antes.
	A memória é do tipo buffer, ou seja, linear e simples, sem lógica de recuperação semântica.
* `from langchain.agents import initialize_agent, AgentType`

 	initialize_agent: função que cria um agente LangChain completo, juntando LLM, memória e ferramentas.
 	AgentType: enumeração que define o tipo de raciocínio que o agente vai usar.
    Exemplo: CHAT_CONVERSATIONAL_REACT_DESCRIPTION cria um agente que age por etapas e lembra do histórico.
* `import google.generativeai as genai`

    Este é o pacote oficial do Google para trabalhar com o Gemini diretamente, sem LangChain.
	Aqui, você usa genai.configure() para informar a chave da API.
	Mesmo que LangChain use sua própria integração, o código precisa configurar a chave com esta biblioteca.
* `import os`

 	Biblioteca padrão do Python para acessar variáveis do sistema operacional.
 	Usado aqui para pegar a chave da API guardada em uma variável de ambiente: os.getenv("GEMINI_API_KEY")

* `from dotenv import load_dotenv`

    Permite carregar variáveis de ambiente definidas em um arquivo .env, que geralmente fica na raiz do projeto.

    Exemplo do arquivo .env:
    GEMINI_API_KEY=chave_super_secreta


In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType
import google.generativeai as genai
import os
from dotenv import load_dotenv

## **Configuração do Gemini**

* `api_key`: busca a chave da API do Gemini do `.env`.
* `temperature=0.7`: controla a **criatividade** da resposta (0 = conservador, 1 = criativo).
* `model="gemini-2.0-flash"`: versão rápida do modelo do Google.


In [25]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=api_key
)

## **Prompt do sistema**
Esse é o **manual do personagem** da IA: o “tutor”. Ele será usado para definir o comportamento inicial do agente.


In [26]:
system_prompt_text = '''
Você é um tutor de Inteligência Artificial amigável e experiente.
Explique conceitos de forma clara e concisa, adapte-se ao nível do aluno
e sempre ofereça o próximo passo lógico no aprendizado.
'''

## ** Memória da conversa**
Essa célula cria a "memória de curto prazo" da IA. Ela armazena as mensagens trocadas.


In [27]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## **Inicialização do agente**
Aqui você junta tudo:
* `llm=chat`: o modelo Gemini.
* `memory`: memória da conversa.
* `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION`: tipo de agente que **pensa em etapas**.
* `verbose=True`: imprime os passos internos (útil para debugging).
* `prefix`: texto do tutor (prompt do sistema).


In [28]:
agent = initialize_agent(
    llm=chat,
    tools=[],
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs={
        "prefix": system_prompt_text
    }
)

## **Pergunta do aluno**
Você envia uma **pergunta de aluno**, e o `agent` responde com base no comportamento de tutor.

In [29]:
pergunta = "Qual a diferença de langchain para langgraph?"
resposta = agent.run(pergunta)
print(resposta)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "LangChain e LangGraph são ambos frameworks para construir aplicações com modelos de linguagem, mas eles abordam o problema de maneiras diferentes. LangChain é um framework mais geral, oferecendo uma ampla gama de ferramentas e componentes para construir diferentes tipos de aplicações LLM, incluindo cadeias (chains), agentes e recuperação de informação. LangGraph, por outro lado, é projetado especificamente para construir aplicações LLM que requerem fluxos de trabalho mais complexos e com estado (stateful). Ele permite definir um grafo de passos, onde cada passo pode ser um modelo de linguagem, uma ferramenta ou outra função. LangGraph é mais adequado para aplicações que precisam rastrear o estado ao longo do tempo e tomar decisões com base nesse estado, como agentes conversacionais complexos ou fluxos de trabalho de decisão automatizados. Em resumo, LangChain é mais versátil e oferece u

## **Criando o juiz**
Um **segundo modelo** é criado, com `temperature=0.3`, ou seja, **menos criativo**, mais crítico e analítico — ideal para atuar como juiz.

In [30]:
juiz = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key=api_key
)

## **Prompt do juiz**
Esse texto é o **manual do juiz**. Ele explica os critérios de avaliação e a estrutura da resposta que ele deve dar.


In [31]:
prompt_juiz = '''
Você é um avaliador imparcial. Sua tarefa é revisar a resposta de um tutor de IA para uma pergunta de aluno.

Critérios:
- A resposta está tecnicamente correta?
- Está clara para o nível médio técnico?
- O próximo passo sugerido está bem formulado?

Se a resposta for boa, diga “✅ Aprovado” e explique por quê.
Se tiver problemas, diga “⚠️ Reprovado” e proponha uma versão melhorada.
'''

## **Função de avaliação**
* `SystemMessage`: define o comportamento do juiz.
* `HumanMessage`: insere o contexto da pergunta e da resposta.
* `juiz.invoke(...)`: faz o juiz **avaliar a resposta** gerada pelo tutor.


In [32]:
from langchain.schema import HumanMessage, SystemMessage

def avaliar_resposta(pergunta, resposta_tutor):
    mensagens = [
        SystemMessage(content=prompt_juiz),
        HumanMessage(content=f"Pergunta do aluno: {pergunta}\n\nResposta do tutor: {resposta_tutor}")
    ]
    return juiz.invoke(mensagens).content

## **Rodando a avaliação**
Você chama a função de avaliação passando a pergunta e a resposta do tutor, e imprime a análise feita pelo juiz.


In [33]:
avaliacao = avaliar_resposta(pergunta, resposta)
print("Avaliação do juiz:\n", avaliacao)

Avaliação do juiz:
 ✅ Aprovado.

A resposta está correta, clara e concisa. Ela explica bem a diferença entre Langchain e LangGraph, e indica quando usar cada um.
